In [1]:
#importing necessary libraries 
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import yfinance as yf
from urllib.parse import quote
import seaborn as sns #library not from class
%matplotlib inline
from scipy.optimize import minimize

In [2]:
#Load .env environment variables
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
# Set a random seed for reproducibility
np.random.seed(40)

In [4]:
# Set Alpaca API key and secret
APCA_API_KEY_ID = os.getenv("APCA_API_KEY")
APCA_API_SECRET_KEY = os.getenv("APCA_SECRET_KEY")
ALPACA_ENDPOINT_KEY = os.getenv("ALPACA_END_POINT")

#ensuring api keys are correct 
#print(os.getenv("APCA_API_KEY_ID"))
#print(os.getenv("APCA_API_SECRET_KEY"))
#print(os.getenv("ALPACA_ENDPOINT_KEY"))

# Create the Alpaca API object
alpaca = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, api_version="v2",base_url= ALPACA_ENDPOINT_KEY)

## Part 1 - Portfolio Optimization 


In [34]:
#initializing variables
initial_investment = 10000
print(f'The initial investment is: ${initial_investment}')

stock_tickers = ['AAPL', 'AMZN', 'MSFT'] 
ticker_string = ', '.join(stock_tickers)
print(f'The selected stocks are: {stock_tickers}')

start_date = '2020-11-08'  # Set your desired start date
print(f'The start date is: {start_date}')

end_date = '2023-11-08'  # Set your desired start date
print(f'The start date is: {start_date}')

num_sims = 500 
print(f'The number of monte carlo simulations is: {num_sims}')

num_years = 3

The initial investment is: $10000
The selected stocks are: ['AAPL', 'AMZN', 'MSFT']
The start date is: 2020-11-08
The start date is: 2020-11-08
The number of monte carlo simulations is: 500


### Import S&P500 Data From Alpacas and Display Closing Prices From Chosen Stocks

In [6]:
from urllib.parse import quote

# Importing SP500 
spy = ["SPY"]

# Define the list of stock tickers, excluding SP500 initially
tickersList = stock_tickers + ['SPY']

# Set timeframe to '1D'
timeframe = '1D'

# Convert start_date and end_date to ISO format with New York timezone
start_date = pd.Timestamp(start_date, tz='America/New_York').isoformat()
end_date = pd.Timestamp(end_date, tz='America/New_York').isoformat()

# Create an empty DataFrame to store the results for tickers
df_combined_tickers = pd.DataFrame()

# Make the request to Alpaca API for each ticker
for ticker in tickersList:
    # Convert the ticker to URL-encoded format
    ticker_encoded = quote(ticker)
    
    # Make the request to Alpaca API to get bars data
    df_ticker = alpaca.get_bars(ticker_encoded, timeframe, limit=None, start=start_date, end=end_date).df
    
    # Select only the 'close' column and assign the new column name with the ticker symbol for clarity purposes
    df_ticker = df_ticker['close'].rename(ticker)
    
    # Combine the results
    df_combined_tickers = pd.concat([df_combined_tickers, df_ticker], axis=1)
    
spyDF = df_combined_tickers['SPY']
df_combined_tickers.drop(columns='SPY',inplace=True)

# Display the results (closing prices for chosen tickers)
df_combined_tickers

,AAPL,AMZN,MSFT
2020-11-09 05:00:00+00:00,116.32,3143.74,218.39
2020-11-10 05:00:00+00:00,116.00,3035.02,211.01
2020-11-11 05:00:00+00:00,119.49,3137.39,216.55
2020-11-12 05:00:00+00:00,119.21,3110.28,215.44
2020-11-13 05:00:00+00:00,119.26,3128.81,216.51
...,...,...,...
2023-11-02 04:00:00+00:00,177.57,138.07,348.32
2023-11-03 04:00:00+00:00,176.65,138.60,352.80
2023-11-06 05:00:00+00:00,179.23,139.74,356.53
2023-11-07 05:00:00+00:00,181.82,142.71,360.53


### Calculate Daily Return, Mean, Standard Deviation, and Last Day Closing Prices for selected tickers and SP500

In [10]:
# Calculate daily returns for all ticker's closing prices
daily_returns_tickers = df_combined_tickers.pct_change()
daily_returns_tickers.dropna(inplace=True)

# Display daily returns for the tickers
print("\nDaily Returns for Chosen Stocks:")
print(daily_returns_tickers)

mean = {} 

# Iterate through each stock in the list of tickers
for stock in stock_tickers:
    # Check if the stock exists in the daily returns columns
    if stock in daily_returns_tickers.columns:
        mean[stock] = daily_returns_tickers.mean()[stock]  # Calculate mean for the stock and store in the dictionary
        print(f'{stock} mean: {mean[stock]}')  # Print the mean of daily returns for the stock


Daily Returns for Chosen Stocks:
                               AAPL      AMZN      MSFT
2020-11-10 05:00:00+00:00 -0.002751 -0.034583 -0.033793
2020-11-11 05:00:00+00:00  0.030086  0.033730  0.026255
2020-11-12 05:00:00+00:00 -0.002343 -0.008641 -0.005126
2020-11-13 05:00:00+00:00  0.000419  0.005958  0.004967
2020-11-16 05:00:00+00:00  0.008720  0.000719  0.003325
...                             ...       ...       ...
2023-11-02 04:00:00+00:00  0.020693  0.007810  0.006502
2023-11-03 04:00:00+00:00 -0.005181  0.003839  0.012862
2023-11-06 05:00:00+00:00  0.014605  0.008225  0.010573
2023-11-07 05:00:00+00:00  0.014451  0.021254  0.011219
2023-11-08 05:00:00+00:00  0.005885 -0.004415  0.007406

[754 rows x 3 columns]
AAPL mean: 0.0007580392192901268
AMZN mean: -0.0011414444220746664
MSFT mean: 0.0008290250824961204


In [11]:
# Calculate and display daily returns for 'SPY'
daily_returns_spy = spyDF.pct_change()
daily_returns_spy.dropna(inplace=True)

# Display daily returns for 'SPY'
print("\nDaily Returns for SPY:")
print(daily_returns_spy)

# Calculate and display mean of daily return for 'SPY'
mean_spy = daily_returns_spy.mean()
print("\nSPY mean:")
print(mean_spy)


Daily Returns for SPY:
2020-11-10 05:00:00+00:00   -0.001523
2020-11-11 05:00:00+00:00    0.007456
2020-11-12 05:00:00+00:00   -0.009475
2020-11-13 05:00:00+00:00    0.013471
2020-11-16 05:00:00+00:00    0.012510
                               ...   
2023-11-02 04:00:00+00:00    0.019164
2023-11-03 04:00:00+00:00    0.009123
2023-11-06 05:00:00+00:00    0.002300
2023-11-07 05:00:00+00:00    0.002846
2023-11-08 05:00:00+00:00    0.000732
Name: SPY, Length: 754, dtype: float64

SPY mean:
0.00033975975529095334


In [12]:
# Calculate and display standard deviation for each ticker
std_devs = {}
for stock in stock_tickers:
    if stock in daily_returns_tickers.columns:
        std_devs[stock] = daily_returns_tickers.std()[stock]
        print(f'{stock} standard deviation: {std_devs[stock]}')

AAPL standard deviation: 0.0177839401327225
AMZN standard deviation: 0.04183369731744983
MSFT standard deviation: 0.017581619238321928


In [13]:
# Calculate and display standard deviation for 'SPY'
std_dev_spy = daily_returns_spy.std()
print("\nStandard Deviation for SPY:")
print(std_dev_spy)


Standard Deviation for SPY:
0.011131185108019043


In [14]:
# Get the last day's closing prices for each ticker
last_day_closing_prices = df_combined_tickers.iloc[-1]

# Display the last day's closing prices
print("\nLast Day's Closing Prices:")
print(last_day_closing_prices)



Last Day's Closing Prices:
AAPL    182.89
AMZN    142.08
MSFT    363.20
Name: 2023-11-08 05:00:00+00:00, dtype: float64


In [15]:
# Get the last day's closing price for 'SPY'
spy_last_day_closing_price = spyDF.iloc[-1]
print("\nLast Day's Closing Price for SPY:")
print(spy_last_day_closing_price)


Last Day's Closing Price for SPY:
437.25


## Part 2 - Monte Carlo Simulations

In [35]:
# Calculate the total number of simulations based on the specified number of years and trading days per year
num_sims = (num_years * 252)  

# Initialize DataFrames to store Monte Carlo simulation results
monte_carlo_results = pd.DataFrame()
monte_carlo_spy_results = pd.DataFrame()

# Create lists and variables to store optimization results and simulated data
optimal_weights_list = []  # Keep optimal weights for later calculation of average weights
simulated_returns_list = []  # Save daily returns of simulated prices for later use

# Create dictionaries and DataFrames to store simulated prices for individual stocks and SPY
simulated_prices_stocks = {stock: pd.DataFrame() for stock in stock_tickers}
simulated_prices_spy = pd.DataFrame()

# Monte Carlo simulation loop
for simulation_index in range(num_sims):
    # Initialize simulated prices lists with the last closing prices
    simulated_prices_dict = {stock: [last_day_closing_prices[stock]] for stock in stock_tickers}
    simulated_prices_spy_list = [spy_last_day_closing_price]

    # Simulate prices for each trading day
    for day_index in range(num_sims):
        for stock in stock_tickers: 
            # Simulate stock prices using a normal distribution based on historical mean and standard deviation
            simulated_price = simulated_prices_dict[stock][-1] * (1 + np.random.normal(mean[stock], std_devs[stock]))
            simulated_prices_dict[stock].append(simulated_price)

        # Simulate SPY prices using a normal distribution based on historical mean and standard deviation
        simulated_spy_price = simulated_prices_spy_list[-1] * (1 + np.random.normal(mean_spy, std_dev_spy))
        simulated_prices_spy_list.append(simulated_spy_price)

    # Store simulated prices in DataFrames
    for stock in stock_tickers:  
        simulated_prices_stocks[stock][simulation_index] = pd.Series(simulated_prices_dict[stock])
    simulated_prices_df = pd.DataFrame(simulated_prices_dict)
    simulated_prices_stocks[simulation_index] = simulated_prices_df
    
    # Calculate daily returns for individual stocks and SPY
    simulated_returns = simulated_prices_df.pct_change()
    simulated_returns.dropna(inplace=True)

    simulated_spy_series = pd.Series(simulated_prices_spy_list)
    simulated_spy_returns = simulated_spy_series.pct_change()
    simulated_spy_returns.dropna(inplace=True)

    # Store daily returns for later analysis
    simulated_returns_list.append(simulated_returns)

    # Define the negative Sharpe ratio function for optimization
    def negative_sharpe_ratio(weights):
        weights_array = np.array(weights)
        portfolio_returns = np.sum(simulated_returns.mean() * weights_array) * 252
        portfolio_volatility = np.sqrt(np.dot(weights_array.T, np.dot(simulated_returns.cov() * 252, weights_array)))
        sharpe_ratio = portfolio_returns / portfolio_volatility
        return -sharpe_ratio

    # Define the constraint function ensuring allocation sums to 1
    def check_allocation_sum(weights):
        return np.sum(weights) - 1

    # Define optimization constraints and boundaries
    constraints = ({'type': 'eq', 'fun': check_allocation_sum})
    allocation_bounds = [(0, 1)] * len(stock_tickers)  

    # Initialize weights with an even distribution
    initial_guess = [1 / len(stock_tickers)] * len(stock_tickers)  

    # Perform portfolio optimization using the scipy minimize function
    optimization_results = minimize(negative_sharpe_ratio, initial_guess, method='SLSQP', bounds=allocation_bounds, constraints=constraints)

    # Retrieve and store optimal weights
    optimal_weights = optimization_results.x
    optimal_weights_list.append(optimal_weights)
    print(optimal_weights)

    # Calculate portfolio daily returns using the optimized weights
    portfolio_daily_returns = simulated_returns.dot(optimal_weights)

    # Calculate cumulative returns for the portfolio and SPY
    monte_carlo_results[simulation_index] = (1 + portfolio_daily_returns.fillna(0)).cumprod()
    monte_carlo_spy_results[simulation_index] = (1 + simulated_spy_returns.fillna(0)).cumprod()


[0.75417884 0.00455754 0.24126362]
[1.04056750e-01 6.67868538e-17 8.95943250e-01]
[7.40608788e-01 7.33497464e-14 2.59391212e-01]
[3.65643241e-01 1.38479722e-16 6.34356759e-01]
[4.00109052e-01 6.51041721e-15 5.99890948e-01]
[9.06823326e-01 7.70053400e-12 9.31766743e-02]
[0.34714745 0.         0.65285255]
[0.59979623 0.         0.40020377]
[0.09691705 0.140092   0.76299095]
[0.332903   0.00295786 0.66413914]
[0.70158067 0.         0.29841933]
[0.44423852 0.         0.55576148]
[0.6685834  0.07306185 0.25835475]
[0.1835018 0.        0.8164982]
[0.52770121 0.         0.47229879]
[0.44032623 0.0453905  0.51428328]
[1.14455909e-01 2.00317193e-15 8.85544091e-01]
[1.00000000e+00 1.64920161e-14 0.00000000e+00]
[1.00000000e+00 1.66533454e-16 3.88578059e-16]
[9.15483486e-01 8.45165135e-02 1.21594184e-16]
[1.50630287e-01 1.47532811e-15 8.49369713e-01]
[4.13526671e-01 3.84071230e-16 5.86473329e-01]
[0.07831991 0.07418116 0.84749893]
[4.26855163e-01 5.73144837e-01 1.51788304e-17]
[7.96992914e-01 3.4

/var/folders/bk/8p802pg50f31n1clyfc4y7700000gn/T/ipykernel_38616/4207142269.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  simulated_prices_stocks[stock][simulation_index] = pd.Series(simulated_prices_dict[stock])
/var/folders/bk/8p802pg50f31n1clyfc4y7700000gn/T/ipykernel_38616/4207142269.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  monte_carlo_results[simulation_index] = (1 + portfolio_daily_returns.fillna(0)).cumprod()
/var/folders/bk/8p802pg50f31n1clyfc4y7700000gn/T/ipykernel_38616/4207142269.py:82: Performanc

[0.32848371 0.         0.67151629]
[0.00000000e+00 1.11022302e-16 1.00000000e+00]
[0.27994912 0.         0.72005088]
[2.39263451e-01 5.20417043e-17 7.60736549e-01]
[2.22044605e-16 0.00000000e+00 1.00000000e+00]
[0.7002749 0.        0.2997251]
[0.82075373 0.         0.17924627]
[2.75436532e-01 4.00892384e-12 7.24563468e-01]
[4.00617226e-16 1.00000000e+00 0.00000000e+00]
[8.08655807e-01 1.71087103e-16 1.91344193e-01]
[5.81534602e-01 1.14476571e-14 4.18465398e-01]
[0.64138557 0.18960701 0.16900742]
[0.8184777 0.        0.1815223]
[0.53521245 0.         0.46478755]
[0.         0.05258327 0.94741673]
[0.90597042 0.         0.09402958]
[0.25209192 0.07374674 0.67416133]
[0.54357239 0.         0.45642761]
[4.97158574e-01 1.85786302e-15 5.02841426e-01]
[5.13326703e-01 4.02887359e-14 4.86673297e-01]
[0.00000000e+00 6.10622664e-16 1.00000000e+00]
[0.79752116 0.07576018 0.12671866]
[0.58703915 0.         0.41296085]
[4.39482737e-01 6.41847686e-17 5.60517263e-01]
[0.9543908 0.        0.0456092]
[3

###  Representing the Simulated Cumulative Returns of a portfolio over time for 500 Monte Carlo Simulations

Each column corresponds to a specific simulation, and each row represents the cumulative returns at a specific time point

In [30]:
monte_carlo

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
1,1.000744,1.003804,1.006329,0.984103,0.988295,0.995355,1.002612,0.981896,0.982345,1.010886,...,0.982948,1.023470,1.004138,0.987465,0.996527,1.000983,1.044351,1.019571,1.000091,0.977434
2,1.019828,1.029030,1.001661,0.975394,0.962674,0.984982,0.984774,0.995450,1.000379,0.997276,...,0.968698,1.035457,1.024151,0.994104,0.999300,1.016300,1.064813,1.014775,0.989218,0.982431
3,1.022801,1.012558,1.016565,0.978533,0.953486,1.005632,0.981688,0.978709,1.022923,1.016050,...,0.990379,1.028450,1.025362,1.006158,1.013308,1.017020,1.042518,1.007104,0.976572,1.002034
4,1.008197,1.031968,1.006551,0.958108,0.972442,0.980418,0.970576,0.975667,0.987576,1.011910,...,1.001216,1.036114,1.039083,1.017414,0.998587,1.008911,1.046984,0.985187,0.960314,0.972270
5,1.007762,1.038569,0.993792,0.956463,0.977967,0.984944,0.957737,0.982916,0.981646,1.003928,...,0.999846,1.045893,1.039117,1.013239,1.000201,1.006534,1.065984,1.008497,0.971635,0.996254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,1.718267,2.060173,1.856330,1.349737,2.211852,1.539497,2.814753,1.776234,2.995561,1.770388,...,2.268870,2.479313,3.252519,2.220345,1.663741,2.871315,1.994668,1.587416,1.800936,1.333157
753,1.741192,2.088790,1.850394,1.324236,2.224214,1.530895,2.801892,1.752299,3.018392,1.797829,...,2.246218,2.513601,3.301965,2.257586,1.675208,2.858127,1.966925,1.546826,1.765353,1.376220
754,1.747582,2.042434,1.834679,1.333380,2.200920,1.525685,2.744519,1.730072,3.052295,1.753651,...,2.233194,2.506176,3.321974,2.317283,1.666108,2.912009,1.944724,1.525761,1.764064,1.374083
755,1.749149,2.049915,1.849660,1.287275,2.232786,1.505720,2.717898,1.711866,3.048789,1.776861,...,2.196030,2.518838,3.358166,2.332038,1.672386,2.885819,1.923333,1.528828,1.740634,1.357839


In [33]:
#commenting out because the output is long (500 sims)
#mc_opt_weights

In [38]:
# Initialize lists to store individual DataFrames
monte_carlo_avg_optimized_list = []
mc_uniform_portfolio_daily_returns_list = []

# Monte Carlo Simulation Loop
for x in range(min(num_simulations, len(simulated_dr_all))):
    # Retrieve Simulated Daily Returns
    simulated_daily_returns = simulated_dr_all[x]
    
    # Calculate Portfolio Daily Returns using Average Weights
    portfolio_daily_returns_optimized = simulated_daily_returns.dot(average_weights)
    
    # Calculate Portfolio Daily Returns using Uniform Weights
    portfolio_daily_returns_uniform = simulated_daily_returns.dot(uniform_weights)
    
    # Create a new DataFrame for each iteration
    df_optimized = pd.DataFrame((1 + portfolio_daily_returns_optimized.fillna(0)).cumprod(), columns=[x])
    df_uniform = pd.DataFrame((1 + portfolio_daily_returns_uniform.fillna(0)).cumprod(), columns=[x])
    
    # Append DataFrames to the lists
    monte_carlo_avg_optimized_list.append(df_optimized)
    mc_uniform_portfolio_daily_returns_list.append(df_uniform)

# Concatenate DataFrames to create the final result
monte_carlo_avg_optimized = pd.concat(monte_carlo_avg_optimized_list, axis=1)
mc_uniform_portfolio_daily_returns = pd.concat(mc_uniform_portfolio_daily_returns_list, axis=1)


### Displaying the Average Weights and Standard Deviation of the Optimized Weights for each stock in the portfolio

In [42]:
# Calculate average weights and standard deviation
average_weights_optimized = np.mean(mc_opt_weights, axis=0)
std_dev_weights_optimized = np.std(mc_opt_weights, axis=0)

# Display results
for i in range(len(stock_tickers)):
    print(f"{stock_tickers[i]}: Average Weight = {average_weights_optimized[i]}, Standard Deviation = {std_dev_weights_optimized[i]}")

AAPL: Average Weight = 0.47206838339940854, Standard Deviation = 0.2865418640778612
AMZN: Average Weight = 0.03450389156985226, Standard Deviation = 0.10602084726212506
MSFT: Average Weight = 0.49342772503009874, Standard Deviation = 0.28676673364374017
